In [57]:
import aocd
data = aocd.get_data(day=1, year=2023)
T = data.split()

trans = {'one':'o1e', 'two':'t2o', 'three':'3e', 'four':'4',
         'five':'5e', 'six':'6', 'seven':'7n', 'eight':'e8t',
         'nine':'n9e'}

def tr(x):
    for s in trans: x = x.replace(s, trans[s])
    return x

def cal(x):
    l = [int(c) for c in x if c.isdigit()]
    return 10*l[0]+l[-1]

print("Part1:", sum(map(cal, T)), "Part2:",sum(map(cal, map(tr,T))))

Part1: 54388 Part2: 53515


In [106]:
import aocd
from collections import defaultdict
data = aocd.get_data(day=2, year=2023)

part1 = part2 = 0

for i, g in enumerate(data.split("\n")):
    _,l = g.split(": ")
    p1 = i+1
    maxes = defaultdict(int)
    for u in l.split("; "):
        for t in u.split(", "):
            n, c = t.split(" ")
            n = int(n)
            if not((c == "red" and n <= 12) 
                or (c == "green" and n <= 13) 
                or (c == "blue" and n <= 14)): p1 = 0
            maxes[c] = max(maxes[c], n)
    part1 += p1
    p2 = 1
    for k in maxes: p2 *= maxes[k]
    part2 += p2
print(part1,part2)

2795 75561


In [107]:
import aocd
data = aocd.get_data(day=3, year=2023)

mp = defaultdict(str)
T = data.split('\n')
n = len(T)
m = len(T[0])

for i,l in enumerate(T):
    for j,c in enumerate(l):
        if c != '.': 
            mp[(i,j)] = c

part1 = part2 = 0
gears = defaultdict(list)
tmp, ispart, cgear = 0, False, set()

for i in range(n):
    for j in range(m):
        if mp[(i,j)].isdigit():
            tmp = tmp*10+int(mp[(i,j)])
            for k in range(-1,2):
                for l in range(-1,2):
                    if mp[(i+k,j+l)] != '' and not(mp[(i+k,j+l)].isdigit()):
                        ispart = True
                    if mp[(i+k,j+l)] == '*': cgear.add((i+k,j+l))           
        else:
            if ispart:
                part1 += tmp
                for g in cgear: gears[g].append(tmp)
            tmp, ispart, cgear = 0, False, set()
    if ispart: 
        part1 += tmp
        for g in cgear: gears[g].append(tmp)
    tmp, ispart, cgear = 0, False, set()

for g in gears:
    if len(gears[g]) == 2:
        part2 += gears[g][0]*gears[g][1]
print("part1:",part1,"part2:",part2)

part1: 535351 part2: 87287096


In [108]:
import aocd
data = aocd.get_data(day=4, year=2023).splitlines()

part1, part2, n, cards = 0, 0, len(data), [1]*len(data)

for i, row in enumerate(data):
    win, hand = row.split(':')[1].split("|")
    match = len(set(win.split()).intersection(hand.split()))
    if match >= 1: part1 += 2**(match-1)
    for j in range(i+1, min(i+1+match, n)): cards[j] += cards[i]
    part2 += cards[i]

print("part1:", part1, "part2:", part2)

part1: 18653 part2: 5921508


In [109]:
import aocd
data = aocd.get_data(day=5, year=2023).split("\n\n")

seeds = list(map(int,data[0].split()[1:]))
part1 = [(seed,seed+1) for seed in seeds]
part2 = [(seeds[2*i],seeds[2*i]+seeds[2*i+1])
         for i in range(len(seeds)//2)]

def desc2trans(desc): return [ (src,src+length,dest-src)
           for line in desc.split('\n')[1:]
           for dest,src,length in [map(int,line.split())]]

def step(s2,transitions):
    ret = []
    for sta,sto in s2:
        for start, stop, delta in transitions:
            if start <= sta < stop:
                ret.append((sta+delta,min(sto,stop)+delta))
                sta, sto = min(sto,stop), sto
        if sta<sto: ret.append((sta,sto))
    return ret

for desc in data[1:]:
    transitions = sorted(desc2trans(desc))
    part1 = step(part1,transitions)
    part2 = step(part2,transitions)

print("part1:",min(part1)[0],"part2:",min(part2)[0])

part1: 199602917 part2: 2254686


In [110]:
import aocd, math, time

data = aocd.get_data(day=6, year=2023).split('\n')
races = list(zip(*map(lambda line: list(map(int,line.split()[1:])), data)))
t,d = map(lambda line: int(line.split(":")[1].replace(" ","")), data)

def win(t,d): return(t-1-int((t-math.sqrt(t*t -4*d))/2)*2)

def win_bf(t,d): return sum(1 for ti in range(t) if ti*(t-ti)>d )

print("Root calculation")
start_time = time.time()
print("part1:", math.prod(win(t,d) for t,d in races), "part2:", win(t,d))
print("--- %s seconds ---" % ((time.time() - start_time)))


Root calculation
part1: 2756160 part2: 34788142
--- 0.0 seconds ---


In [111]:
import aocd
from collections import Counter

data = aocd.get_data(day=7, year=2023).split('\n')

def parse(s, part):
    hand, bid = s.split()
    bid,cnt = int(bid), Counter(hand)
    cards = ['23456789TJQKA', 'J23456789TQKA']
    
    if part == 2: js, cnt['J'] = cnt['J'], 0
    else: js = 0

    v = list(sorted(cnt.values(), reverse=True))
    v[0] += js
    return (v,tuple(map(cards[part-1].index,hand)),bid)

for part in [1,2]:
    T = sorted(map(lambda l:parse(l,part),data))
    res = sum( i*bid for i,(_,_,bid) in enumerate(T,start=1) )
    print("Part"+str(part)+":",res)

Part1: 251058093
Part2: 249781879


In [112]:
import aocd,math,itertools
path, _, *data = aocd.get_data(day=8, year=2023).split('\n')
graph = {l[:3]:(l[7:10],l[12:15]) for l in data}

def gdepth(nodes):
    for i in itertools.count(1):
        nodes = { dest for node in nodes for dest in graph[node] }
        if any(node[-1]=='Z' for node in nodes): return i

cycles = {node:gdepth({node})*len(path) for node in graph if node[-1] == 'A'}

print('Part1:', cycles['AAA'], 'Part2', math.lcm(*cycles.values()))

Part1: 17621 Part2 20685524831999


In [113]:
import aocd
from more_itertools import dotproduct

data = aocd.get_data(day=9, year=2023).split('\n')
data = list(map(lambda x: list(map(int,x.split())),data))

lg = [len(data[0])]
for i in range(2,lg[0]+1): lg.append(-lg[-1]*(lg[0]-i+1)//i)

print("Part1:", sum(dotproduct(l[::-1],lg) for l in data),
      "Part2:", sum(dotproduct(l,lg) for l in data))

Part1: 1684566095 Part2: 1136


In [114]:
import aocd
from itertools import product

def offset(pos,di): return (pos[0]+dirs[di][0],pos[1]+dirs[di][1])

dirs = [(1,0),(0,1),(-1,0),(0,-1)]
letters = "-F7LJ|S."
maps = [{0,2},{0,1},{1,2},{0,3},{2,3},{1,3},{0,1,2,3},{}]
parity = [1,0.5,-0.5,-0.5,0.5,0]

data = aocd.get_data(day=10, year=2023).split('\n')
m,n = len(data), len(data[0])
s = letters.index('S')

sparse = {(i,j):letters.index(data[j][i]) for i,j in product(range(n),range(m))}
neighbours = {pos:{offset(pos, di) for di in maps[idx]} for pos, idx in sparse.items()}
S = next(pos for pos in sparse if sparse[pos] == s)

for next_s in neighbours[S]:
    pos, path, next_pos = S, {S}, next_s
    while next_pos in sparse and pos in neighbours[next_pos] and sparse[next_pos] < 6:
        path.add(next_pos)
        pos, next_pos = next_pos, next(p for p in neighbours[next_pos] if p!=pos)
    if next_pos == S:
        sparse[S] = next( k for k in range(1,len(maps))
                       if {next_s, pos} == {offset(S,di) for di in maps[k]})
        break

part2 = s = 0
for i,j in product(range(m),range(n)):
    if (i,j) in path: s += parity[sparse[(i,j)]]
    else: part2+= int(s)%2

print("Part1:", len(path)//2, "Part2:", part2)

Part1: 6599 Part2: 477


In [115]:
import aocd

data = aocd.get_data(day=11, year=2023).split('\n')
n,m = len(data), len(data[0])

lines = [ (i,c) for i in range(n) for c in [data[i].count('#')] if c>0 ]
rows = [ (j,c) for j in range(m) 
        for c in [sum(data[i][j]=="#" for i in range(n))] if c>0 ]

def rawgap(li,raw=0,gap=0):
    total = serie = li[0][1]
    prev =  li[0][0]
    prev_gaps = 0
    for i,cnt in li[1:]:
        raw += cnt*serie
        prev_gaps += (i-prev-1)*total
        gap += cnt*prev_gaps
        total += cnt
        serie += total
        prev = i
    return raw,gap

raw,gap = rawgap(rows,*rawgap(lines))

print("Part1: %d Part2: %d" % (raw+gap*2, raw+gap*10**6))



Part1: 9734203 Part2: 568914596391


In [116]:
import aocd

def parseline(line):
    m,l = line.split()
    return(m,tuple(map(int,l.split(','))))

data = list(map(parseline, aocd.get_data(day=12, year=2023).split('\n')))

cache = {}
def dp(S,L):
    if (S,L) in cache: return(cache[(S,L)])
    s,l = len(S), len(L)
    if s < sum(L)+l : return 0
    if l == 0: return all(c in '?.' for c in S)
    ret = dp(S[1:], L) if S[0] in '.?' else 0
    if all(c in '?#' for c in S[:L[0]]) and (s<L[0] or S[L[0]] in '.?'):
        ret += dp(S[L[0]+1:], L[1:])
    cache[(S,L)] = ret
    return ret

print("Part1: %d Part2: %d" % (sum(dp(s+".",l) for s,l in data), 
                               sum(dp((s+'?')*5,l*5)  for s,l in data)))

Part1: 7307 Part2: 3415570893842


In [117]:
import aocd
data = aocd.get_data(day=13, year=2023)
patts = [line.split() for line in data.split("\n\n")]

def day13(pat, s, rot):
    if rot: pat = tuple(zip(*pat))
    return(next((i for i in range(1,len(pat))
                if sum(t!=b for lt,lb in zip(pat[i-1::-1],pat[i:])
                       for t,b in zip(lt,lb)) == s),0))
for s in range(2):
    print("Part%d: %d"% ((s+1,sum(day13(pat, s, True)+
                                  100*day13(pat, s, False)
                                  for pat in patts))))



Part1: 30518
Part2: 36735


In [61]:
import aocd

data = aocd.get_data(day=14, year=2023).split("\n")
n = len(data)
m = len(data[0])
sparse = {(i,j):data[i][j] for i in range(n) for j in range(m)}

def load(hs,n,m): return sum(n-i for i,_ in hs)

def rot(sparse,n,m): return {(j,n-i-1):sparse[(i,j)] 
                             for i,j in sparse},m,n

def slide(sparse,n,m):
    cur = [0]*m
    for j in range(m):
        for i in range(n):
            if (i,j) in sparse:
                if sparse[(i,j)] == 'O':
                    del sparse[(i,j)]
                    sparse[(cur[j],j)] = 'O'
                    cur[j] += 1
                elif sparse[(i,j)] == '#':
                    cur[j] = i+1

def fullrot(sparse, n, m):
    for _ in range(4):
        slide(sparse,n,m)
        sparse,n,m = rot(sparse,n,m)
    return sparse

def hash(sparse): return tuple(sorted(k for k in sparse if sparse[k] == 'O'))

slide(sparse,n,m)
print("Part1:", load(hash(sparse),n,m))

cache = {hash(sparse):0}
for c in range(1,10**9):
    sparse = fullrot(sparse, n, m)
    hs = hash(sparse)
    if hs in cache:
        start, length = cache[hs],c-cache[hs]
        val = start+(1000000000-start)%length
        print("Part2:",load(next(k for k in cache if cache[k] == val),n,m))
        break
    cache[hs] = c

Part1: 107142
Part2: 104815


In [63]:
import aocd

data = [[c for c in line] for line in aocd.get_data(day=14, year=2023).split("\n")]
n,m = len(data), len(data[0])

def slidenorth(data):
    for j in range(m):
        c = 0
        for i in range(n):
            if data[i][j] == 'O': data[c][j],data[i][j],c = data[i][j], data[c][j], c+1
            elif data[i][j] == '#': c = i+1
    return data

def slidesouth(data):
    for j in range(m):
        c = n-1
        for i in range(n-1,-1,-1):
            if data[i][j] == 'O': data[c][j],data[i][j],c = data[i][j], data[c][j], c-1
            elif data[i][j] == '#': c = i-1
    return data

def slideeast(data):
    for i in range(n):
        c = m-1
        for j in range(m-1,-1,-1):
            if data[i][j] == 'O': data[i][c],data[i][j],c = data[i][j], data[i][c], c-1
            elif data[i][j] == '#': c = j-1
    return data

def slidewest(data):
    for i in range(n):
        c = 0
        for j in range(m):
            if data[i][j] == 'O': data[i][c],data[i][j],c = data[i][j],data[i][c],c+1
            elif data[i][j] == '#': c = j+1
    return data

def load(hs): return sum(n-i for (i,j) in hs)
def fullrot(data): return slideeast(slidesouth(slidewest(slidenorth(data))))
def hash(data): return tuple((i,j) for i in range(n) for j in range(m) if data[i][j] == 'O')

slidenorth(data)
print("Part1:", load(hash(data)))

cache = {hash(data):0}
for c in range(1,10**9):
    data = fullrot(data)
    hs = hash(data)
    if hs in cache:
        start, length = cache[hs],c-cache[hs]
        val = start+(1000000000-start)%length
        print("Part2:",load(next(k for k in cache if cache[k] == val)))
        break
    cache[hs] = c

Part1: 107142
Part2: 104815


In [119]:
import aocd
from functools import reduce

data = aocd.get_data(day=15, year=2023)
def hiter(c,ch): return (c+ord(ch))*17%256
def hash(s): return reduce(hiter,s,0)

print("Part1", sum(hash(s) for s in data.split(',')))

boxes = [dict() for _ in range(256)]
for s in data.split(','):
    label = s.strip('-').split('=')
    box = boxes[hash(label[0])]
    if len(label) == 1: box.pop(label[0],0)
    else: box[label[0]] = int(label[1])
        
print("Part2", sum(i*j*f for i,b in enumerate(boxes, 1)
                for j,f in enumerate(b.values(), 1)))

Part1 497373
Part2 259356


In [120]:
import aocd

dirs = [(0,1),(-1,0),(0,-1),(1,0)]
ndirs = {('/'):[[1],[0],[3],[2]], ('\\'):[[3],[2],[1],[0]],
         ('-'):[[0],[0,2],[2],[0,2]],('|'):[[1,3],[1],[1,3],[3]],
         ('.'):[[0],[1],[2],[3]]}
def tadd(a,b): return (a[0]+b[0],a[1]+b[1])

data = aocd.get_data(day=16, year=2023).split('\n')
n,m = len(data), len(data[0])
sparse = { (i,j):data[i][j] for i in range(n) for j in range(m)}

def dfs(pos,di):
    energized = {pos}
    beams = [(pos,ndir) for ndir in ndirs[sparse[pos]][di]]
    visited = set(beams)
    while len(beams):
        pos,di = beams.pop()
        npos = tadd(pos,dirs[di])
        if npos in sparse:
            if npos not in energized: energized.add(npos)
            for ndir in ndirs[sparse[npos]][di]:
                if (npos,ndir) not in visited:
                    visited.add((npos,ndir))
                    beams.append((npos,ndir))
    return len(energized)

print("Part1: %d Part2: %d" % (dfs((0,0),0),
      max(dfs((i,j),d) for d in range(4) 
          for i in (range(n) if d%2==0 else [0,n-1])
          for j in (range(m) if d%2==1 else [0,m-1]))))


Part1: 7199 Part2: 7438


In [6]:
from heapq import heappush, heappop
import aocd
data = aocd.get_data(day=17, year=2023).split('\n')
sparse = {(i,j):int(c) for i,line in enumerate(data) for j,c in enumerate(line)}
n,m = len(data), len(data[0])
def tadd(a,b): return (a[0]+b[0],a[1]+b[1])
dirs = [(1,0), (0,-1), (-1,0), (0,1)]

def solve(mi,ma):
    heap = [(0,(0,0),(0,0))]
    visited = {((0,0),(0,0))}
    while len(heap):
        cost, pos, lsdir = heappop(heap)
        for di in range(-1,2):
            if di == 0:
                if lsdir[1] == ma: continue
                else: idx = lsdir[1]+1
            else:
                idx = 1
                if 0< lsdir[1] < mi: continue
            ndi = (lsdir[0]+di)%4
            npos = tadd(pos, dirs[ndi])
            if (npos in sparse) and ((npos,(ndi,idx)) not in visited):
                ncost = cost + sparse[npos]
                if npos == (n-1,m-1): return ncost
                visited.add((npos,(ndi,idx)))
                heappush(heap,(ncost,npos,(ndi,idx)))

print("Part1: %d Part2: %d" % (solve(0,3),solve(4,10)))

Part1: 847 Part2: 997


In [22]:
from heapq import heappush, heappop, heapify
import aocd
data = aocd.get_data(day=17, year=2023).split('\n')
#print(data[0])
sparse = {(i,j):int(c) for i,line in enumerate(data) for j,c in enumerate(line)}
n,m = len(data), len(data[0])
def tadd(a,b): return (a[0]+b[0],a[1]+b[1])
def taddm(a,b,c): return (a[0]+b[0]*c,a[1]+b[1]*c)
dirs = [(1,0), (0,-1), (-1,0), (0,1)]

def solve(mi,ma):
    heap = [(0,(0,0),0),(0,(0,0),3)]
    heapify(heap)
    visited = set()
    ret = 1000
    while len(heap):
        cost, pos, di = heappop(heap)
        if (pos, di) in visited : continue
        visited.add((pos, di))
        if cost>ret: break
        dir1 = (di+1)%4
        dir2 = (di-1)%4
        for step in range(ma):
            pos = tadd(pos,dirs[di]) 
            if pos not in sparse: break
            cost += sparse[pos]
            if step+1 >= mi:
                if pos == (n-1,m-1): 
                    ret = min(ret, cost)
                if (pos,dir1) not in visited:
                    heappush(heap,(cost,pos,dir1))
                    heappush(heap,(cost,pos,dir2))
    return ret
print("Part1: %d Part2: %d" % (solve(1,3),solve(4,10)))

Part1: 847 Part2: 997


In [55]:
import aocd
data = aocd.get_data(day=17, year=2023).split('\n')
sparse = {(i,j):int(c) for i,line in enumerate(data) for j,c in enumerate(line)}
n,m = len(data), len(data[0])
def tadd(a,b): return (a[0]+b[0],a[1]+b[1])
dirs = [(1,0), (0,-1), (-1,0), (0,1)]

def solve(mi,ma):
    ha = [[] for _ in range((n+m)*9)]
    visited = [False for _ in range(n*m*4)]
    ha[0] = [((0,0),0),((0,0),3)]
    ret = 1000
    for cost, bucket in enumerate(ha):
        for pos, di in bucket:
            dcost = 0
            if visited[pos[0]*m+pos[1]+di*n*m] : continue
            visited[pos[0]*m+pos[1]+di*n*m] = True
            if cost+mi>ret: break
            dir1 = (di+1)%4
            dir2 = (di-1)%4
            for step in range(ma):
                pos = tadd(pos,dirs[di]) 
                if pos not in sparse: break
                dcost += sparse[pos]
                if step+1 >= mi:
                    if pos == (n-1,m-1):
                        ret = min(ret, cost+dcost)
                    if visited[pos[0]*m+pos[1]+dir1*n*m] == False:
                        ha[cost+dcost].append((pos,dir1))
                        ha[cost+dcost].append((pos,dir2))
    return ret
print("Part1: %d Part2: %d" % (solve(1,3),solve(4,10)))

Part1: 847 Part2: 997


In [30]:
import aocd
from collections import defaultdict
data = aocd.get_data(day=17, year=2023).split('\n')
sparse = {(i,j):int(c) for i,line in enumerate(data) for j,c in enumerate(line)}
n,m = len(data), len(data[0])

def tadd(a,b): return (a[0]+b[0],a[1]+b[1])
def taddm(a,b,c): return (a[0]+b[0]*c,a[1]+b[1]*c)
dirs = [(1,0), (0,-1), (-1,0), (0,1)]

def solve(mi,ma):
    ha = defaultdict(list)
    ha[0] = [((0,0),0),((0,0),3)]
    visited = set()
    ret = 10000
    cost = 0
    while True:
        for pos, di in ha[cost]:
            dcost = 0
            if (pos, di) in visited : continue
            visited.add((pos, di))
            if cost+mi>ret: return ret
            dir1 = (di+1)%4
            dir2 = (di-1)%4
            for step in range(ma):
                pos = tadd(pos,dirs[di]) 
                if pos not in sparse: break
                dcost += sparse[pos]
                if step+1 >= mi:
                    if pos == (n-1,m-1):
                        ret = min(ret, cost+dcost)
                    if (pos,dir1) not in visited:
                        ha[cost+dcost].append((pos,dir1))
                        ha[cost+dcost].append((pos,dir2))
        cost += 1

print("Part1: %d Part2: %d" % (solve(1,3),solve(4,10)))

Part1: 847 Part2: 997


In [51]:
# With thanks to azzal07, who adapted my previous bucket queue solution to massively improve
# runtime. By:
# - Translating the grid into 1D coordinates (and making neighbors match with faster
#   math). It's very clever how (+/-) next_d = w//d gives the cw and ccw turns!
# - inlining a fixed size bucket queue inside the dijkstra impl.
# - using a fixed list for dijkstra mins instead of a dict.
# - unrolling neighbors in the hot loop
import aocd

D = [i.strip() for i in aocd.get_data(day=17, year=2023).split('\n')]

g = [int(ch) for row in D for ch in row]
h = len(D)
w = len(D[0])
end = len(g) - 1
MAX_SIZE = w * h * 9


def dynamic_dijkstra(minimum, maximum, end):
    seen, mins = set(), [[MAX_SIZE] * (w + 2) for _ in range(w * h)]
    bh = [[] for _ in range(MAX_SIZE)]
    for state in (0, 1), (0, w):
        for c, (yx, d) in neighbors(state, minimum, maximum):
            mins[yx][d] = c
            bh[c].append((yx, d))
    for cost, vs in enumerate(bh):
        for v in vs:
            if v in seen:
                continue

            if v[0] == end:
                return cost

            seen.add(v)

            yx, d = v
            next_d = w//d
            next_cost = cost
            for l in range(1, maximum):
                yx += d
                if yx < 0 or end < yx or (d == 1 and yx % w == 0) or (d == -1 and yx % w == (w-1)):
                    break

                next_cost += g[yx]
                if l >= minimum:
                    m = mins[yx]
                    if next_cost < m[next_d]:
                        m[next_d] = next_cost
                        bh[next_cost].append((yx, next_d))
                    if next_cost < m[-next_d]:
                        m[-next_d] = next_cost
                        bh[next_cost].append((yx, -next_d))


def neighbors(state, minimum, maximum):
    yx, d = state
    cc = 0
    for l in range(1, maximum):
        yx += d
        if yx < 0 or end < yx or (d == 1 and yx % w == 0) or (d == -1 and yx % w == (w-1)):
            break

        cc += g[yx]
        if l >= minimum:
            yield (cc, (yx, -w//d))
            yield (cc, (yx, w//d))



import time
START = time.time_ns()
print("day17", dynamic_dijkstra(1, 4, end), dynamic_dijkstra(4, 11, end))
print(">>>", (time.time_ns()-START)/1e9)

day17 847 997
>>> 2.1947282


In [122]:
import aocd
data = aocd.get_data(day=18, year=2023)
dirs = {'D':(1,0),'L':(0,-1),'U':(-1,0),'R':(0,1)}
dirs2 = [(1,0),(0,1),(-1,0),(0,-1)]
def tadd(a,b,c=1): return (a[0]+b[0]*c,a[1]+b[1]*c)
points = [ (d,int(n),c[2:-1],int(c[2:-2],16),int(c[-2:-1])) for line in data.split('\n') for d,n,c in [line.split()]]

prev1, prev2 = (0,0), (0,0)
area1=area2=peri1=peri2= 0

def newpoint(steps,di,pos,peri,area):
    npos = tadd(pos,di,steps)
    return npos, peri+steps, area + pos[0]*npos[1] - pos[1]*npos[0]

for point in points:
    prev1,peri1,area1 = newpoint(point[1],dirs[point[0]],prev1,peri1,area1)
    prev2,peri2,area2 = newpoint(point[3],dirs2[point[4]],prev2,peri2,area2)

print ("Part1: %d Part2: %d" % (((abs(area1)+peri1) // 2) + 1, ((abs(area2)+peri2) // 2) + 1))

Part1: 39194 Part2: 78242031808225


In [123]:
import aocd, math

data = aocd.get_data(day=19, year=2023)
W,P = data.split("\n\n")
P = [{ k:int(v) for item in part[1:-1].split(",") for k,v in [item.split("=")]} 
     for part in P.split('\n')]
W = { k:[ c.split(":") for c in v[:-1].split(",")] 
     for w in W.split("\n") for k,v in [w.split("{")]}

part1 = 0
for p in P:
    cur = 'in'
    while cur not in ['A','R']:
        cur = next(( c[1] for c in W[cur][:-1] if eval(c[0], {}, p)),W[cur][-1][0])
    if cur == 'A': part1 += sum(p.values())

def countparts(di): return math.prod(di[k][1]-di[k][0]+1 for k in di)

def part2(w,di):
    if w == "A": return countparts(di)
    elif w == "R": return 0
    else:
        di = {k:v for k,v in di.items()}
        ret = 0
        for c in W[w][:-1]:
            p, op, val = c[0][0], c[0][1], int(c[0][2:])
            start,stop = di[p]
            if op == "<":
                if start<val:
                    di[p] = (start,min(stop,val-1))
                    ret += part2(c[1],di)
                if val<=stop:
                    di[p] = (max(start,val),stop)
                else: return ret
            else:
                if stop>val:
                    di[p] = (max(start,val+1),stop)
                    ret += part2(c[1],di)
                if val>=start:
                    di[p] = (start,min(stop,val))
                else: return ret
        ret += part2(W[w][-1][0],di)
        return ret
    
print("Part1: %d Part2: %d" % (part1, part2('in',{k:(1,4000) for k in 'xmas'})))

Part1: 432788 Part2: 142863718918201


In [124]:
import aocd
from math import lcm

data = aocd.get_data(day=20, year=2023)
modules = { s[1:]:(s[0],d.split(', ')) for line in data.split("\n") for s,d in [line.split(" -> ")]}
mem = {}
for m in modules:
    if modules[m][0] == '%':
        mem[m] = False
    for d in modules[m][1]:
        if d in modules and modules[d][0] == '&':
            if d in mem: mem[d][m] = False
            else: mem[d] = {m:False}

def flow(s,d,ishigh):
    if modules[d][0] == "%":
        if ishigh: return []
        else: ne = mem[d] = not(mem[d])
    else:
        mem[d][s] = ishigh
        ne = any(not(v) for v in mem[d].values())
    return [(m, ne) for m in modules[d][1]]

def button(cpt):
    fifo = [('roadcaster',d,False) for d in modules['roadcaster'][1]]
    cpt[0] += 1
    while len(fifo):
        s,d,ishigh = fifo.pop(0)
        li = flow(s,d,ishigh)
        for nd,nishigh in li:
            cpt[nishigh] += 1
            if nd == 'lv' and nishigh and d not in part2: part2[d] = i
            if nd in modules: fifo.append((d,nd,nishigh))
    return cpt
cpt, i, part2 = [0,0], 1, {}
for i in range(1,10000):
    cpt = button(cpt)
    if i == 1000: print("Part1: ", cpt[0]*cpt[1])
    if len(part2) == 4:
        print("Part2: ", lcm(*(part2.values())))
        break

Part1:  634473756
Part2:  233338595643977


In [21]:
import aocd
data = aocd.get_data(day=21, year=2023).split('\n')
n = len(data)
sparse = {(i,j) for i in range(n) for j in range(n) if data[i][j] in '.S'}
S = next((i,j) for i in range(n) for j in range(n) if data[i][j] == 'S')
dirs = [(1,0),(-1,0),(0,1),(0,-1)]

def tadd(a,b): return ((a[0]+b[0]),(a[1]+b[1]))
def modp(a): return(a[0]%n, a[1]%n)

visited, new, cache = {S}, {S}, {0:1}
k, r  = 26501365//n, 26501365%n

for c in range(1,r+2*n+1):
    new,visited = { np for p in new for di in dirs for np in [tadd(p,di)] 
           if np not in visited and modp(np) in sparse}, new
    cache[c] = len(new) + (cache[c-2] if c>1 else 0)

d2 = cache[r+2*n]+cache[r]-2*cache[r+n]
d1 = cache[r+2*n]-cache[r+n]
print("Part1: %d Part2: %d" % (cache[60],cache[r+2*n]+(k-2)*(2*d1+(k-1)*d2)//2))

Part1: 3261 Part2: 627960775905777


In [4]:
[0,1,2,3,4,5][4::-1]

[4, 3, 2, 1, 0]

In [126]:
import aocd

data = aocd.get_data(day=22, year=2023).split('\n')
heights, supported = [], []
L = [[int(c) for c in line.replace('~',',').split(',')] for line in data]
L.sort(key=lambda x:x[2])

for i,b in enumerate(L):
    B = list(((b2[5]-b2[2])+heights[j]+1,j) 
             for j,b2 in enumerate(L[:i])
             if b2[3]>=b[0] and b2[0]<=b[3] and b2[4]>=b[1] and b2[1]<=b[4])
    heights.append(max((b[0] for b in B),default=1))
    supported.append([j for h,j in B if h == heights[i]])

unremovable = {v for l in supported if len(l)==1 for v in l}

part2 = 0
for i in unremovable:
    dis = {i}
    for j in range(i+1,len(L)):
        if len(supported[j]) and all(s in dis for s in supported[j]): dis.add(j)
    part2 += len(dis)-1
    
print("Part1: %d Part2: %d" % (len(L) - len(unremovable),part2))

Part1: 524 Part2: 77070


In [127]:
import aocd

data = aocd.get_data(day=23, year=2023).split("\n")
n, m = len(data), len(data[0])

def tadd(a,b): return (a[0]+b[0], a[1]+b[1])
dirs = [(0,1),(0,-1),(1,0),(-1,0)]
slopes = [".><",".<>",".v^",".^v"]

M = {(i,j):c for i,line in enumerate(data) for j, c in enumerate(line)}
S = (0,next(i for i,c in enumerate(data[0]) if c=='.'))
E = (n-1,next(i for i,c in enumerate(data[-1]) if c=='.'))

oneway,oneway2,border, graph, idx = set(),set(),{0},{},0

def graphadd(graph, s, d, cost):
    if s not in graph: graph[s] = {}
    graph[s][d] = cost

visited, nm = {S}, {S:0}
stack = [(S,i) for i,di in enumerate(dirs) if tadd(S,di) in M and M[tadd(S,di)] in '.<>^v']
while len(stack):
    start,direction = stack.pop(0)
    prev, cpos, le = start, tadd(start,dirs[direction]), 1
    visited.add(cpos)
    way = slopes[direction].index(M[cpos])
    ndirs = [(i,npos in visited) for i,di in enumerate(dirs)
             for npos in [tadd(cpos,di)] if npos in M and M[npos] != '#']
    while len(ndirs) == 2:
        direction = next(di for di,_ in ndirs if prev != tadd(cpos,dirs[di]))
        prev, cpos, le = cpos, tadd(cpos,dirs[direction]), le+1
        way |= slopes[direction].index(M[cpos])
        visited.add(cpos)
        ndirs = [(i,npos in visited) for i,di in enumerate(dirs)
                 for npos in [tadd(cpos,di)] if npos in M and M[npos] != '#']
    
    if cpos not in nm:
        idx += 1
        nm[cpos] = idx
        
    if way == 1: oneway.add((nm[cpos], nm[start]))
    else: oneway.add((nm[start], nm[cpos]))

    graphadd(graph, nm[start], nm[cpos], le)
    graphadd(graph, nm[cpos], nm[start], le)

    if len(ndirs) == 3 and nm[start] in border:
        border.add(nm[cpos])
        oneway2.add((nm[cpos],nm[start]))

    for di in [di2 for di2,vis in ndirs if not(vis)]: stack.append((cpos,di))

def longuestpath(oneway):
    fwd, lmax = {}, 0

    stack = [(0,0,0,0)]
    while len(stack):
        p,l,v,d = stack.pop()
        if d == (idx) // 2:
            if p not in fwd: fwd[p] = []
            fwd[p].append((l,v))
            continue
        for n in graph[p]:
            if (p,n) in oneway or v & 1<<n: continue
            stack.append((n,l+graph[p][n],v|1<<p,d+1))
    for n in fwd: fwd[n].sort(reverse=True)
    
    stack = [(nm[E],0,0,0)]
    while len(stack):
        p,l,v,d = stack.pop()
        if d == (idx+1) // 2:
            continue
        if p == 0: 
            lmax = max(lmax, l)
            continue
        if p in fwd:
            for fl,fv in fwd[p]:
                if lmax > l+fl: break
                if (v|1<<p) & fv: continue
                lmax = max(lmax, l+fl)
        for n in graph[p]:
            if (n,p) in oneway or v & 1<<n: continue
            stack.append((n,l+graph[p][n],v|1<<p,d+1))
    return lmax

print("Part1:", longuestpath(oneway), "Part2:", longuestpath(oneway2))

Part1: 2166 Part2: 6378


In [128]:
import aocd, numpy as np
from itertools import combinations

data = aocd.get_data(day=24, year=2023)

L = [ list(map(int,l.replace(' @',',').split(', '))) for l in data.split('\n')]
mi, ma, part1 = 200000000000000, 400000000000000, 0

for a,b in combinations(L,2):
    if (b[4]/b[3] - a[4]/a[3]) == 0: continue
    x = (a[1]-b[1] + b[4]*b[0]/b[3] - a[4]*a[0]/a[3])/(b[4]/b[3] - a[4]/a[3])
    t1, t2 = (x-a[0])/a[3] , (x-b[0])/b[3]
    y = a[1]+a[4]*t1
    part1 += (t1>=0 and t2>=0 and mi<=x<=ma and mi<=y<=ma)

print("Part1: ", part1)

x1,x2,x3,x4 = L[:4]

a = np.array([[(x2[4] - x1[4]), (x1[3] - x2[3]),0,(x1[1] - x2[1]),(x2[0] - x1[0]),0],
            [(x2[5] - x1[5]),0, (x1[3] - x2[3]),(x1[2] - x2[2]),0,(x2[0] - x1[0])],
            [(x3[4] - x1[4]), (x1[3] - x3[3]),0,(x1[1] - x3[1]),(x3[0] - x1[0]),0],
            [(x3[5] - x1[5]),0, (x1[3] - x3[3]),(x1[2] - x3[2]),0,(x3[0] - x1[0])],
            [(x4[4] - x1[4]), (x1[3] - x4[3]),0,(x1[1] - x4[1]),(x4[0] - x1[0]),0],
            [(x4[5] - x1[5]),0, (x1[3] - x4[3]),(x1[2] - x4[2]),0,(x4[0] - x1[0])]])
b = np.array([-x1[0]*x1[4] + x2[0]*x2[4] + x1[1]*x1[3] - x2[1]*x2[3],
            -x1[0]*x1[5]+x2[0]*x2[5]+x1[2]*x1[3]-x2[2]*x2[3],
            -x1[0]*x1[4]+x3[0]*x3[4]+x1[1]*x1[3]-x3[1]*x3[3],
            -x1[0]*x1[5]+x3[0]*x3[5]+x1[2]*x1[3]-x3[2]*x3[3],
            -x1[0]*x1[4]+x4[0]*x4[4]+x1[1]*x1[3]-x4[1]*x4[3],
            -x1[0]*x1[5]+x4[0]*x4[5]+x1[2]*x1[3]-x4[2]*x4[3]])
x = list(map(round,np.linalg.solve(a, b)))

# answers lack precision, need to round them and rebase from real points
t1 = round((x[0]-x1[0]) / (x[3]-x1[3]))
print("Part2: ",sum(x1[:3])-t1*(sum(x1[3:])-sum(x[3:])))

Part1:  19523
Part2:  566373506408017


In [129]:
from collections import defaultdict
from heapq import heappop, heappush
from itertools import pairwise

data = aocd.get_data(day=25, year=2023)
M = defaultdict(set)
for line in data.split("\n"):
    src,dst = line.split(': ')
    for de in dst.split():
        M[src].add(de)
        M[de].add(src)

def bfs(start, exclusions = {}, stop = -1):
    visited = {start:(0,[start])}
    heap = [(0,start,[start])]
    while len(heap)>0:
        dist, node, path = heappop(heap)
        for de in M[node]:
            if (node,de) in exclusions: continue
            if de not in visited:
                visited[de] = (dist+1, path+[de])
                if de == stop and False:
                    return len(visited),visited,stop
                heappush(heap,(dist+1,de,path+[de]))
    return (len(visited),visited, node)

def findcut():
    start = next(k for k in M)
    _,visited,stop = bfs(start)
    p1 = list(pairwise(visited[stop][1]))
    p1 = p1[len(p1)//2-1::-1]+p1[len(p1)//2:]
    for i,(s,d) in enumerate(p1):
        exclusions = {(s,d),(d,s)}
        _,visited2,_ = bfs(start, exclusions,stop)
        p2 = list(pairwise(visited2[stop][1]))
        p2 = p2[len(p2)//2-1::-1]+ p2[len(p2)//2:]
        for j, (s2,d2) in enumerate(p2):
            if (s2,d2) in p1: continue
            exclusions = {(s,d),(d,s), (s2,d2),(d2,s2)}
            _,visited3,_ = bfs(start, exclusions,stop)
            p3 = list(pairwise(visited3[stop][1]))
            p3 = p3[len(p3)//2-1::-1]+ p3[len(p3)//2:]
            for k,(s3,d3) in enumerate(p3):
                if (s3,d3) in p1 or (s3,d3) in p2: continue
                exclusions = {(s,d),(d,s), (s2,d2),(d2,s2), (s3,d3),(d3,s3)}
                lena,_,nstop = bfs(start, exclusions,stop)
                if len(M) != lena:
                   return (lena*(len(M)-lena))

print("AoC 2023:", findcut())


AoC 2023: 606062
